In [1]:
import quippy
import ase
from ase.calculators.calculator import all_changes

import logging
logging.basicConfig(level=logging.WARNING)

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import os
import sys

In [4]:
sys.path.insert(0,"/home/es732/programs/function_modules/")
import mbx_functions

In [5]:
dirpath = os.getcwd()
print dirpath

/home/es732/programs/mbx_tests


In [6]:
from collections import OrderedDict

In [7]:
d_list=[0.1,0.01,0.001,0.0001,0.00001,0.000001]

In [8]:
version_list=["mbx_pbc","mbx_pbc_no_ww_www_co2ww","mbx_pbc_no_co2w"]

In [9]:
dict_version_d_virial=OrderedDict([(version,
                                    OrderedDict([(d, [[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                                 for d in d_list]))
                                   for version in version_list])

In [10]:
dict_version_model_virial=OrderedDict([(version,[[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                   for version in version_list])

In [11]:
def calculate_numerical_virial_diag(pot, in_atoms, d=1e-2):
    ## modified from the function calculate_numerical_stress() at ase/calculators/calculator.py 
    """Calculate numerical stress using finite difference."""

    atoms=in_atoms.copy()
    stress = np.zeros((3, 3), dtype=float)

    cell = atoms.cell.copy()
    V = atoms.get_volume()
    for i in range(3):
        x = np.eye(3)
        x[i, i] += d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eplus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        x[i, i] -= 2 * d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eminus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        stress[i, i] = (eplus - eminus) / (2 * d * V)
        x[i, i] += d
    atoms.set_cell(cell, scale_atoms=True)
    virial= (-stress*atoms.get_volume())
    return virial

In [12]:
this_root="/home/es732/programs/test_genice/CO2/sI/"

In [13]:
this_struct="sI_co2_cell2x2x2_rand0_ase.xyz"
this_at_tmp=quippy.Atoms(this_root+this_struct)
dict_pot=mbx_functions.create_mbx_potential_variables_co2_h2o(this_at_tmp)

In [14]:
np.sum(this_at_tmp.numbers==8)-64*2+64

432

In [15]:
dict_pot

{'n_monomers_types': '368 OHH 64 COO', 'nmon': 432}

## full MBX

In [16]:
this_version=version_list[0]
print this_version

mbx_pbc


In [17]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [18]:
at= this_at_tmp.copy()

In [19]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[ 1.47201270e+02  2.80973830e-02  1.46323351e-02]
 [ 2.80973830e-02  1.46872093e+02 -1.29849463e-01]
 [ 1.46323351e-02 -1.29849463e-01  4.13338055e+02]]


In [20]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## diagonal_virial=F, no PIPs except mw 2B

In [21]:
this_version=version_list[1]
print this_version

mbx_pbc_no_ww_www_co2ww


In [22]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [23]:
at= this_at_tmp.copy()

In [24]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-5.26705602e+02  2.40198305e-01  4.29459541e-01]
 [ 2.40198305e-01 -5.27674169e+02 -1.98667968e+00]
 [ 4.29459541e-01 -1.98667968e+00 -2.59694841e+02]]


In [25]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## full MBX - no 2B CO2 - water PIP

In [16]:
this_version=version_list[2]
print this_version

mbx_pbc_no_co2w


In [17]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [18]:
at= this_at_tmp.copy()

In [19]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[ 1.26240897e+02  1.17775182e-01  3.99142492e-01]
 [ 1.17775182e-01  1.25875417e+02 -3.97774287e-01]
 [ 3.99142492e-01 -3.97774287e-01  3.17334846e+02]]


In [20]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [21]:
version_list

['mbx_pbc', 'mbx_pbc_no_ww_www_co2ww', 'mbx_pbc_no_co2w']

## Errors in derivative virial / Virial from model (only diagonal)

In [29]:
this_version=version_list[0]
print this_version
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

mbx_pbc
d = 0.01
[ 0.02594839  0.0258328  -0.0050292 ]
d = 0.001
[ 0.02186442  0.02181484 -0.01278746]
d = 0.0001
[ 0.02182292  0.02177529 -0.01286562]


In [30]:
this_version=version_list[1]
print this_version
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

mbx_pbc_no_ww_www_co2ww
d = 0.01
[-0.00682638 -0.00677712  0.0088574 ]
d = 0.001
[-0.00610615 -0.00606797  0.02036226]
d = 0.0001
[-0.00609894 -0.00606087  0.02047737]


In [22]:
this_version=version_list[2]
print this_version
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

mbx_pbc_no_co2w
d = 0.01
[0.00423306 0.00415526 0.00942343]
d = 0.001
[4.30507154e-05 4.07830231e-05 9.49324528e-05]
d = 0.0001
[3.86627140e-07 3.75731806e-07 9.31906840e-07]
